In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install -q transformers accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 21.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
job_titles = [
    "Data Scientist",
    "Frontend Developer",
    "Machine Learning Engineer",
    "Software Engineer",
    "AI Researcher"
]

prompt = f"""
You are given a list of job titles.

Task:
1. Rank these job titles based on their relevance to Artificial Intelligence and Data-related careers.
2. Provide ranking from highest priority to lowest priority.
3. Briefly explain your reasoning.

Job Titles:
{job_titles}
"""

messages = [
    {
        "role": "system",
        "content": (
            "You are an expert career analysis assistant. "
            "You are good at evaluating job relevance and ranking job roles logically."
        )
    },
    {
        "role": "user",
        "content": prompt
    }
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)


### Ranking Based on Relevance to Artificial Intelligence and Data-Related Careers

1. **AI Researcher**
2. **Machine Learning Engineer**
3. **Data Scientist**
4. **Software Engineer**
5. **Frontend Developer**

### Reasoning:

1. **AI Researcher**: This role is directly focused on advancing the field of artificial intelligence through research, development, and experimentation with AI technologies. It is highly relevant as it involves core aspects of AI such as machine learning, deep learning, and natural language processing.

2. **Machine Learning Engineer**: This role involves applying machine learning techniques to solve real-world problems. Machine learning is a subset of AI, and this position is crucial for developing and deploying AI systems, making it very relevant to both AI and data-related careers.

3. **Data Scientist**: While not solely focused on AI, Data Scientists heavily rely on AI and machine learning techniques to analyze and interpret complex data. They use algorith

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Apziva/ProjectC.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [ ]:
job_titles = df.iloc[:, 1].tolist()
print(job_titles)

['2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Native English Teacher at EPIK (English Program in Korea)', 'Aspiring Human Resources Professional', 'People Development Coordinator at Ryan', 'Advisory Board Member at Celal Bayar University', 'Aspiring Human Resources Specialist', 'Student at Humber College and Aspiring Human Resources Generalist', 'HR Senior Specialist', 'Student at Humber College and Aspiring Human Resources Generalist', 'Seeking Human Resources HRIS and Generalist Positions', 'Student at Chapman University', 'SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR', 'Human Resources Coordinator at InterContinental Buckhead Atlanta', '2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', '2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Native En

In [ ]:
def run_llm(model, tokenizer, messages, max_tokens=100):

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_tokens
    )

    # skip input
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    return response

In [ ]:
def build_score_prompt(title):
    role_keywords = ["Aspiring Human Resources", "seeking human resources"]
    prompt = f"""
    You are a professional career ranking expert.

    Your task is to evaluate candidates based on their job title.
    The role we are hiring for is: Aspiring human resources or seeking human resources.
    Only consider job titles that match this role or are closely related.

    Job titles that are unrelated to the role should get 1.
    Job titles that are slightly related but not ideal should get 2 or 3.
    Only titles that are fully relevant get 4 or 5.

    Examples:
    - "Aspiring Human Resources Professional" -> 5
    - "HR Generalist" -> 4
    - "Business Analyst" -> 2
    - "English Teacher" -> 1

    Evaluate the fit of this job title for the given role and give a score from 1 (low) to 5 (high).
    Only output the score.

    Job Title: {title}
    Role Keywords: {', '.join(role_keywords)}
    """

    messages = [
    {"role": "system", "content": "You are a professional career ranking expert, focusing on assessing candidate fit for a specified role."},
    {"role": "user", "content": prompt}
    ]
    return messages


scores = []
for title in job_titles:
    messages = build_score_prompt(title)
    response = run_llm(model, tokenizer, messages)
    try:
        score = int(response.strip())
    except:
        score = None
    scores.append(score)

df["rough_score"] = scores

print(df)

      id                                          job_title  \
0      1  2019 C.T. Bauer College of Business Graduate (...   
1      2  Native English Teacher at EPIK (English Progra...   
2      3              Aspiring Human Resources Professional   
3      4             People Development Coordinator at Ryan   
4      5    Advisory Board Member at Celal Bayar University   
..   ...                                                ...   
99   100  Aspiring Human Resources Manager | Graduating ...   
100  101              Human Resources Generalist at Loparex   
101  102   Business Intelligence and Analytics at Travelers   
102  103                     Always set them up for Success   
103  104   Director Of Administration at Excellence Logging   

                                location connection  fit  rough_score  
0                         Houston, Texas         85  NaN            5  
1                                 Kanada      500+   NaN            1  
2    Raleigh-Durham, North 

In [ ]:
df_sorted = df.sort_values(by="rough_score", ascending=False)
df_high = df_sorted[df_sorted["rough_score"].isin([5])]

high_score_titles = df_high["job_title"]
print(high_score_titles)
print(len(high_score_titles))

0     2019 C.T. Bauer College of Business Graduate (...
2                 Aspiring Human Resources Professional
6     Student at Humber College and Aspiring Human R...
5                   Aspiring Human Resources Specialist
13    2019 C.T. Bauer College of Business Graduate (...
14    2019 C.T. Bauer College of Business Graduate (...
9     Seeking Human Resources HRIS and Generalist Po...
8     Student at Humber College and Aspiring Human R...
32                Aspiring Human Resources Professional
35                  Aspiring Human Resources Specialist
38    Student at Humber College and Aspiring Human R...
36    Student at Humber College and Aspiring Human R...
20                Aspiring Human Resources Professional
23                  Aspiring Human Resources Specialist
18    2019 C.T. Bauer College of Business Graduate (...
16                Aspiring Human Resources Professional
28    Aspiring Human Resources Management student se...
30    2019 C.T. Bauer College of Business Gradua

In [ ]:
def build_pair_prompt(title_a, title_b, role="Aspiring Human Resources", role_keywords=None):
    """
    construct LLM messages，compare job_titles in pair。
    role: the role we are hiring for
    title_a: job title A
    title_b: job title B
    role_keywords: list[str] keywords that describe the role
    """

    if role_keywords is None:
        role_keywords = ["Aspiring Human Resources", "seeking human resources"]

    prompt = (f'''
        You are a professional career ranking expert.\n"
        The role we are hiring for is: {role}.\n"
        Only consider job titles that match this role or are closely related. "
        Job titles unrelated to this role should be considered less fit.\n\n"
        Compare these two job titles and determine which candidate is more fit for the role.\n"
        A: {title_a}\n"
        B: {title_b}\n"
        Role Keywords: {', '.join(role_keywords)}\n"
        Only answer 'A' if title A is more fit, or 'B' if title B is more fit."
        ''')

    messages = [
        {"role": "system", "content": "You are a professional career ranking assistant, focusing on assessing candidate fit for a specified role."},
        {"role": "user", "content": prompt}
    ]
    return messages

def pairwise_score_sort(titles, role="Aspiring Human Resources", role_keywords=None):
    n = len(titles)
    scores = [0] * n  # title scores when they win

    for i in range(n):
        for j in range(i+1, n):
            messages = build_pair_prompt(titles[i], titles[j], role=role, role_keywords=role_keywords)
            resp = run_llm(model, tokenizer, messages, max_tokens=5)
            resp = resp.strip().upper()  # 确保大写

            if resp == "A":
                scores[i] += 1
            elif resp == "B":
                scores[j] += 1
            else:
                # 如果输出异常或平局，给半分
                scores[i] += 0.5
                scores[j] += 0.5

    # 按分数排序
    sorted_titles = [t for _, t in sorted(zip(scores, titles), key=lambda x: x[0], reverse=True)]
    return sorted_titles, scores

print(len(df_high["job_title"].drop_duplicates()))

high_score_titles = df_high["job_title"].drop_duplicates().head(10).tolist()
sorted_titles, scores = pairwise_score_sort(high_score_titles, role="Aspiring Human Resources")
print(sorted_titles)
print(scores)

15
['2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Aspiring Human Resources Professional', 'Aspiring Human Resources Specialist', 'Aspiring Human Resources Manager | Graduating May 2020 | Seeking an Entry-Level Human Resources Position in St. Louis', 'Aspiring Human Resources Management student seeking an internship', 'Student at Humber College and Aspiring Human Resources Generalist', 'Seeking Human Resources HRIS and Generalist Positions', 'Aspiring Human Resources Manager, seeking internship in Human Resources.', 'Seeking Human Resources Position', 'Nortia Staffing is seeking Human Resources, Payroll & Administrative Professionals!!  (408) 709-2621']
[8.5, 7.5, 4.0, 6.5, 3, 5, 6.5, 1, 3.0, 0]


In [ ]:
df_output = pd.DataFrame({
    "job_title": sorted_titles,
    "pairwise_score": scores
})

# 按 score 降序排序（可选，因为 sorted_titles 已经是排序好的）
df_output = df_output.sort_values(by="pairwise_score", ascending=False).reset_index(drop=True)

print(df_output)

                                           job_title  pairwise_score
0  2019 C.T. Bauer College of Business Graduate (...             8.5
1              Aspiring Human Resources Professional             7.5
2  Aspiring Human Resources Manager | Graduating ...             6.5
3  Seeking Human Resources HRIS and Generalist Po...             6.5
4  Student at Humber College and Aspiring Human R...             5.0
5                Aspiring Human Resources Specialist             4.0
6  Aspiring Human Resources Management student se...             3.0
7                   Seeking Human Resources Position             3.0
8  Aspiring Human Resources Manager, seeking inte...             1.0
9  Nortia Staffing is seeking Human Resources, Pa...             0.0


In [ ]:
How to fine-tune these models
large data sets, machine(Gpu)
instead of from scratch, we fine-tune them effectively with specific data (like company data, recent data ...)
Lora: low-rank adaptation.
Q-lora: less compute ()

Be able to fine-tune





Use case of LLM:
    use llm through api, for free (skip)




In [ ]:
def build_pair_prompt(title_a, title_b, role="Aspiring Human Resources", role_keywords=None):
    """
    construct LLM messages，compare job_titles in pair。
    role: the role we are hiring for
    title_a: job title A
    title_b: job title B
    role_keywords: list[str] keywords that describe the role
    """

    if role_keywords is None:
        role_keywords = ["Aspiring Human Resources", "seeking human resources"]

    prompt = (f'''
        You are a professional career ranking expert.\n"
        The role we are hiring for is: {role}.\n"
        Only consider job titles that match this role or are closely related. "
        Job titles unrelated to this role should be considered less fit.\n\n"
        Compare these two job titles and determine which candidate is more fit for the role.\n"
        A: {title_a}\n"
        B: {title_b}\n"
        Role Keywords: {', '.join(role_keywords)}\n"
        Only answer 'A' if title A is more fit, or 'B' if title B is more fit."
        ''')

    messages = [
        {"role": "system", "content": "You are a professional career ranking assistant, focusing on assessing candidate fit for a specified role."},
        {"role": "user", "content": prompt}
    ]
    return messages

def pairwise_score_sort(titles, role="Aspiring Human Resources", role_keywords=None):
    n = len(titles)
    scores = [0] * n  # title scores when they win

    for i in range(n):
        for j in range(i+1, n):
            messages = build_pair_prompt(titles[i], titles[j], role=role, role_keywords=role_keywords)
            resp = run_llm(model, tokenizer, messages, max_tokens=5)
            resp = resp.strip().upper()  # 确保大写

            if resp == "A":
                scores[i] += 1
            elif resp == "B":
                scores[j] += 1
            else:
                # 如果输出异常或平局，给半分
                scores[i] += 0.5
                scores[j] += 0.5

    # 按分数排序
    sorted_titles = [t for _, t in sorted(zip(scores, titles), key=lambda x: x[0], reverse=True)]
    return sorted_titles, scores

df_high = df_sorted[df_sorted["rough_score"].isin([4,5])]
high_score_titles = df_high["job_title"].drop_duplicates().tolist()
print(len(high_score_titles))
sorted_titles, scores = pairwise_score_sort(high_score_titles, role="Aspiring Human Resources")
print(sorted_titles)
print(scores)

15
['2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Aspiring Human Resources Manager | Graduating May 2020 | Seeking an Entry-Level Human Resources Position in St. Louis', 'Aspiring Human Resources Professional', 'Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment', 'Aspiring Human Resources Specialist', 'Aspiring Human Resources Management student seeking an internship', 'Student at Humber College and Aspiring Human Resources Generalist', 'Aspiring Human Resources Professional | An energetic and Team-Focused Leader', 'Aspiring Human Resources Manager, seeking internship in Human Resources.', 'Seeking Human Resources HRIS and Generalist Positions', 'Human Resources|\nConflict Management|\nPolicies & Procedures|Talent Management|Benefits & Compensation', 'Seeking Human Resources Position', 'Nortia Staffing is seeking Human Resources, Payroll & Administrative Pr